It’s important to understand data types because storing data in the appropriate format is fundamental to building usable databases and performing
accurate analysis.

In a SQL database, each column in a table can hold one and only one data type, which you define in the __CREATE TABLE__  statement by declaring the data type after the column name. 

In [ ]:
CREATE TABLE eagle_watch (
    observation_date date, 
    eagles_seen integer, 
    notes text
);

These data types fall into the three categories you’ll encounter most: 
- __Characters:__ Any character or symbol 
- __Numbers:__ Includes whole numbers and fractions
- __Dates and times:__ Temporal information

## Understanding Characters

Character string types are general-purpose types suitable for any combination of text, numbers, and symbols. Character types include the following:
- __char(n):__ 
    - A fixed-length column where the character length is specified by n. 
- __varchar(n):__ 
    - A variable-length column where the maximum length is specified by n. If you insert fewer characters than the maximum, PostgreSQL will not store extra spaces. For example, the string blue will take four spaces, whereas the string 123 will take three. In large databases, this practice saves considerable space. This type, included in standard SQL, also can be specified using the longer name character varying(n).
- __text:__ 
    - A variable-length column of unlimited length. (According to the PostgreSQL documentation, the longest possible character string you can store is about 1 gigabyte).
    
There is no substantial difference in performance among the three types. The flexibility and potential space savings of varchar and text seem to give them an advantage. But if you search discussions online, some users suggest that defining a column that will always have the same number of characters with char is a good way to signal what data it should contain. For instance, you might see char(2) used for US state postal abbreviations.

CREATE TABLE char_data_types ( 
    char_column char(10), 
    varchar_column varchar(10), 
    text_column text
);
INSERT INTO char_data_types 
    VALUES
        ('abc', 'abc', 'abc'), ('defghi', 'defghi', 'defghi');
COPY char_data_types TO 'C:\YourDirectory\typetest.txt'
WITH (FORMAT CSV, HEADER, DELIMITER '|');

In PostgreSQL, COPY table_name FROM is the import function, and COPY table_name TO is the export function.

## Understanding Numbers

The SQL number types include the following: 
- __Integers:__ Whole numbers, both positive and negative
- __Fixed-point and floating-point:__ Two formats of fractions of whole numbers

### Using Integers

These are whole numbers, both positive and negative, including zero.

The SQL standard provides three integer types: smallint, integer, and bigint. The difference between the three types is the maximum size of the numbers they can hold. 

__Auto-Incrementing Integers__

With PostgreSQL, you have two ways to auto-increment an integer column. One is the serial data type, a PostgreSQL-specific implementation of the ANSI SQL standard for auto-numbered identity columns. The other is
the ANSI SQL standard IDENTITY keyword.

    1. Auto-Incrementing with serial

In [ ]:
    CREATE TABLE people ( 
        id serial,
        person_name varchar(100)
    );

    Every time a new row with a person_name is added to the table, the id column will increment by 1.

2. Auto-Incrementing with IDENTITY

    You can specify IDENTITY in two ways: . 

        1. GENERATED ALWAYS AS IDENTITY tells the database to always fill the column with an auto-incremented value.
        2. GENERATED BY DEFAULT AS IDENTITY tells the database to fill the column with an auto-incremented value by default if the user does not supply one. This option allows for the possibility of duplicate values, which can make use of it problematic for creating key columns.

In [ ]:
        CREATE TABLE people ( 
            id integer GENERATED ALWAYS AS IDENTITY, 
            person_name varchar(100)
        );

For the id data type, we use integer followed by the keywords __GENERATED ALWAYS AS IDENTITY.__ Now, every time we insert a person_name value into the table, the database will fill the id column with an auto-incremented value.

## Using Decimal Numbers

Decimals represent a whole number plus a fraction of a whole number; the fraction is represented by digits following a decimal point. In a SQL database, they’re handled by fixed-point and floating-point data types.

### Understanding Fixed-Point Numbers

The fixed-point type, also called the arbitrary precision type, is numeric(precision,scale). You give the argument precision as the maximum number of digits to the left and right of the decimal point, and the argument scale as the number of digits allowable on the right of the decimal point. Alternately, you can specify this type using decimal(precision,scale). Both are part of the ANSI SQL standard. If you omit specifying a scale value, the scale will be set to zero; in effect, that creates an integer. If you omit specifying the precision and the scale, the database will store values of any precision and scale up to the maximum allowed.

### Understanding Floating-Point Types

The two floating-point types are real and double precision, both part of the SQL standard. The difference between the two is how much data they store. The real type allows precision to six decimal digits, and double precision to 15 decimal digits of precision, both of which include the number of digits on both sides of the point. These floating-point types are also called variable-precision types. The database stores the number in parts representing the digits and an exponent—the location where the decimal point belongs. So, unlike numeric, where we specify fixed precision and scale, the decimal point in a given column can “float” depending on the number.

### Using Fixed- and Floating-Point Types

1. numeric, decimal:Up to 131,072 digits before the decimal point; up to 16,383 digits after the decimal point
2. real: 6 decimal digits precision
3. double precisio: 15 decimal digits precision

In [ ]:
CREATE TABLE number_data_types ( 
    numeric_column numeric(20,5), 
    real_column real,
    double_column double precision );
INSERT INTO number_data_types 
    VALUES
        (.7, .7, .7),
        (2.13579, 2.13579, 2.13579), (2.1357987654, 2.1357987654, 2.1357987654);
        
SELECT * FROM number_data_types;

In [ ]:
numeric_column 
--------------
0.70000 
2.13579 
2.13580

real_column 
-----------
0.7 
2.13579 
2.1357987

double_column 
-------------
0.7 
2.13579
2.1357987654

### Running into Trouble with Floating-Point Math

The storage required by the numeric data type is variable, and depending on the precision and scale specified, numeric can consume considerably more space than the floating-point types. If you’re working with millions of rows, it’s worth considering whether you can live with relatively inexact
floating-point math.

In [ ]:
SELECT  numeric_column * 10000000 AS fixed,
           real_column * 10000000 AS floating 
    FROM number_data_types
WHERE numeric_column = .7;

In [ ]:
fixed 
------------
7000000.00000

floating
----------------
6999999.88079071

### Choosing Your Number Data Type

Use integers when possible. Unless your data uses decimals, stick with integer types. If you’re working with decimal data and need calculations to be exact (dealing with money, for example), choose numeric or its equivalent, decimal. Float types will save space, but the inexactness of floating-point math won’t pass muster in many applications. Use them only when exactness is not as important.

Choose a big enough number type. Unless you’re designing a database to hold millions of rows, err on the side of bigger. When using numeric or decimal, set the precision large enough to accommodate the number of digits on both sides of the decimal point. With whole numbers, use bigint unless you’re absolutely sure column values will be constrained to fit into
the smaller integer or smallint type.

## Understanding Dates and Times

__timestamp__ Records date and time, which are useful for a range of situations you might track: departures and arrivals of passenger flights, a schedule, or incidents along a timeline. You will almost always want to add the keywords with time zone to ensure that the _time recorded_ for an event includes the time zone where it occurred.

__date__ Records just the date. 

__time__ Records just the time. 

__interval__ Holds a value representing a unit of time expressed in the format quantity unit. It doesn’t record the start or end of a time period, only its length. 

In [ ]:
CREATE TABLE date_time_types (
    timestamp_column timestamp with time zone, 
    interval_column interval
);

INSERT INTO date_time_types 
    VALUES
        ('2022-12-31 01:00 EST','2 days'), 
        ('2022-12-31 01:00 -8','1 month'),
        ('2022-12-31 01:00 Australia/Melbourne','1 century'),
        (now(),'1 week');

SELECT * FROM date_time_types;

In [ ]:
timestamp_column 
----------------------------
2022-12-31 01:00:00-05
2022-12-31 04:00:00-05 
2022-12-30 09:00:00-05
2020-05-31 21:31:15.716063-05

interval_column 
---------------
2 days 1 mon
100 years
7 days

## Using the interval Data Type in Calculations

The interval data type is useful for easy-to-understand calculations on date and time data. For example, let’s say you have a column that holds the date a client signed a contract. Using interval data, you can add 90 days to
each contract date to determine when to follow up with the client.

SELECT timestamp_column, 
        interval_column,
    timestamp_column - interval_column AS new_date
FROM date_time_types;

Note that the new_date column by default is formatted as type timestamp with time zone, allowing for the display of time values as well as dates if the interval value uses them.

## Understanding JSON and JSONB

JSON, short for JavaScript Object Notation, is a structured data format used for both storing data and exchanging data between computer systems.

In [ ]:
{   "business_name": "Old Ebbitt Grill", 
    "business_type": "Restaurant", 
    "employees": 300, "address": {
        "street": "675 15th St NW", 
        "city": "Washington",
        "state": "DC", "zip_code": "20005"
}
}

This snippet of JSON shows the format’s basic structure. A key, for example business_name, is associated with a value—in this case, Old Ebbitt Grill. A key also can have as its value a collection of additional key/value pairs, as shown with address.

PostgreSQL currently offers two data types for JSON, which both enforce valid JSON and support functions for working with data in that format:

1. __json__ Stores an exact copy of the JSON text 
2. __jsonb__ Stores the JSON text in a binary format 

There are significant differences between the two. For example, jsonb supports indexing, which can improve processing speed.


## Using Miscellaneous Types


PostgreSQL supports many additional types, including but not limited to the following: 

- __Boolean type__ that stores a value of true or false 
- __Geometric types__ that include points, lines, circles, and other twodimensional objects 
- __Text search types__ for PostgreSQL’s full-text search engine 
- __Network address types__, such as IP or MAC addresses 
- __A universally unique identifier (UUID) type__, sometimes used as a unique key value in tables
- __Range types__, which let you specify a range of values, such as integers or timestamps 
- __Types for storing binary data__
- __XML data type__ that stores information in that structured format

## Transforming Values from One Type to Another with CAST

The __CAST()__ function succeeds only when the target data type can accommodate the original value. Casting an integer as text is possible, because the character types can include numbers. Casting text with letters of the alphabet as a number is not.

In [ ]:
SELECT timestamp_column, CAST(timestamp_column AS varchar(10)) 
FROM date_time_types;

SELECT numeric_column,
    CAST(numeric_column AS integer), 
    CAST(numeric_column AS text)
FROM number_data_types;

SELECT CAST(char_column AS integer) FROM char_data_types;

## Using CAST Shortcut Notation

Insert the double colon in between the name of the column and the data type you want to convert it to. For example, these two statements cast _timestamp_column_ as a _varchar_:

In [ ]:
SELECT timestamp_column, CAST(timestamp_column AS varchar(10))
FROM date_time_types;
S
ELECT timestamp_column::varchar(10)
FROM date_time_types;

## TRY IT YOURSELF

__Continue exploring data types with these exercises:__

__Your company delivers fruit and vegetables to local grocery stores, and you need to track the mileage driven by each driver each day to a tenth of a mile. Assuming no driver would ever travel more than 999 miles in a day, what would be an appropriate data type for the mileage column in your table? Why?__

__numeric(4,1)__

The data type numeric(4,1) consists of four digits in total (the precision) and one digit after the decimal point (the scale). This choice would allow you to store a value as large as 999.9.

The reason for selecting this data type is that it precisely fits the given assumption that no driver would ever travel more than 999 miles in a day. By using four digits for the total precision, we can accommodate any number between 0 and 999, and the one-digit scale after the decimal point allows us to represent the tenth of a mile accurately.

However, it's essential to consider the practical aspect that real-world scenarios might not always align perfectly with assumptions. There could be exceptional cases where a driver's mileage exceeds 999.9 miles in a day. In such cases, the data type "numeric(5,1)" could be more suitable, as it would provide an additional digit for the total precision, allowing you to store values as large as 9999.9 miles.

__In the table listing each driver in your company, what are appropriate data types for the drivers’ first and last names? Why is it a good idea to separate first and last names into two columns rather than having one larger name column?__

__varchar(50)__ or __text__

__varchar(50):__  It is a suitable choice for storing names as it allows you to efficiently use storage space when names are shorter than the maximum limit.
__text:__ The "text" data type is another option, which can store a large amount of character data, making it suitable for longer names or other text-based data. It doesn't have a specific maximum length, which means it can accommodate exceptionally long names if they occur.

_Separating First and Last Names:_

It is a good idea to separate first and last names into two distinct columns for several reasons:

1. Flexibility: Having separate columns for first and last names allows for better flexibility when handling names. You can easily perform operations on each name independently, such as searching, sorting, or filtering based on either the first or last name.

2. Avoiding Data Duplication: Storing full names in a single column might lead to data duplication. For example, if two drivers have the same last name but different first names, you would end up repeating the last name for each entry, which wastes storage space and may make the database harder to maintain.

3. Data Integrity: Splitting names into two columns enhances data integrity. It ensures that each name component is stored separately and reduces the risk of unintentional data corruption or errors.

__Assume you have a text column that includes strings formatted as dates. One of the strings is written as '4//2021'. What will happen when you try to convert that string to the timestamp data type?__

Attempting to convert a string of text that does not conform to accepted date/time formats will result in an error. You can see this with the below example, which tries to cast the string as a timestamp.



In [ ]:
SELECT CAST('4//2021' AS timestamp with time zone);

The correct date format typically includes a day, month, and year separated by appropriate characters like slashes or hyphens, such as '04/2021' or '2021-04-01'. Since '4//2021' does not adhere to this standard date format, the database will raise an error during the conversion attempt.

To successfully convert the string to a timestamp, you would need to ensure that the date string is correctly formatted with all the necessary date components and appropriate separators, adhering to the standard date format recognized by your database.